In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

FILE_URL = 'https://www.eia.gov/opendata/bulk/NG.zip'
SPOT_PRICE_DAILY_ID = 'NG.RNGWHHD.D'
ALL_IDS = [SPOT_PRICE_DAILY_ID, 'NG.NW2_EPG0_SNO_R33_BCF.W', 'NG.NW2_EPG0_SSO_R33_BCF.W', 'NG.NW2_EPG0_SWO_R31_BCF.W', 'NG.NW2_EPG0_SWO_R32_BCF.W', 'NG.NW2_EPG0_SWO_R33_BCF.W', 'NG.NW2_EPG0_SWO_R34_BCF.W', 'NG.NW2_EPG0_SWO_R35_BCF.W',
           'NG.NW2_EPG0_SWO_R48_BCF.W', 'NG.RNGC1.D', 'NG.RNGC2.D', 'NG.RNGC3.D', 'NG.RNGC4.D', 'NG.N9050US1.M', 'NG.N9050US2.M', 'NG.N9140US1.M', 'NG.N9140US2.M', 'NG.N9220US1.M', 'NG.N5020US2.M', 'NG.N9100US2.M', 'NG.N9130US2.M']

## Open the file with info and drop rows without data


In [ ]:
with open("NG.txt", "r") as f:
    df = pd.read_json(f, lines=True)

In [ ]:
df = df[df['data'].notna()]

In [ ]:
df.columns

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df['name']

## Show series which are related to interesting data


In [ ]:
# US NG Production
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(df[(df['name'].str.contains('U.S.')) & (df['name'].str.contains(
        'Production')) & (df['f'].isin(['D', 'W', 'M']))][['name', 'series_id']])

In [ ]:
# Othert interesting series
# & (df['geography2'].isna())
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
#     display(df[(df['name'].str.contains('U.S.')) & (df['f'].isin(['D', 'W', 'M'])) & (df['name'].str.contains('Exports'))][['name', 'series_id']])

## Try to plot data about daily prices during a year


In [ ]:
daily_price = df[df['series_id'] == SPOT_PRICE_DAILY_ID]

In [ ]:
prices = daily_price['data'].tolist()
price_df = pd.DataFrame(data=[price[1] for price in prices[0]], index=[
                        price[0] for price in prices[0]], columns=['price'])

In [ ]:
price_df.index = pd.to_datetime(price_df.index)

In [ ]:
grouped_price_df = price_df.groupby(price_df.index.year)

In [ ]:
def get_prices_for_year(df, year):
    dr = pd.date_range(datetime.datetime(2000, 1, 1),
                       datetime.datetime(2000, 12, 31))
    curr_group = df.get_group(year)
    new_list = []
    for date in dr:
        query = curr_group[(curr_group.index.month == date.month) & (
            curr_group.index.day == date.day)]
        new_list.append(None if query.empty else query['price'].iloc[0])
    return pd.Series(new_list, index=dr, name=year).dropna()

In [ ]:
plt.figure(figsize=(15, 10))
for key in grouped_price_df.groups.keys():
    if key > 2015:
        md_prices = get_prices_for_year(grouped_price_df, key)
        md_prices = md_prices / md_prices.iloc[0]
        # plt.plot(grouped_price_df.get_group(key).index.to_series().apply(lambda s: datetime.date(1, s.month, s.day)), grouped_price_df.get_group(key))
        plt.plot(md_prices, label=str(key))
plt.ylim(0, 3)
plt.axvline(datetime.datetime(2000, 9, 18))
plt.legend()
plt.show()

In [ ]:
all_years = []
years = []
for key in grouped_price_df.groups.keys():
    if key > 2015:
        md_prices = get_prices_for_year(grouped_price_df, key)
        md_prices = md_prices / md_prices.iloc[0]
        all_years.append(md_prices)
        # years.append(key)

all_years_df = pd.concat(all_years, axis=1)

In [ ]:
all_years_df.head()

In [ ]:
all_years_df['mean'] = all_years_df.mean(axis=1, skipna=True)
all_years_df['min'] = all_years_df.min(axis=1, skipna=True)
all_years_df['max'] = all_years_df.max(axis=1, skipna=True)
all_years_df.sort_index()
start = all_years_df.index.searchsorted(datetime.datetime(2000, 4, 1))
end = all_years_df.index.searchsorted(datetime.datetime(2000, 5, 30))

In [ ]:
import matplotlib.dates as mdates
plt.figure(figsize=(15, 10))
all_years_df.plot(y=["min", "mean", "max"], kind="line")
plt.fill_between(all_years_df.index, all_years_df["min"], all_years_df["max"])
plt.ylim(0, 3)
plt.axvline(datetime.datetime(2000, 9, 18), color='black')
plt.legend()
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=7))
plt.show()

##


In [ ]:
# Wyłapywać wyjątki i poszukać co tam jest źle
all_data_list = []
for name in df['name']:
    data = df[df['name'] == name]['data'].to_list()
    try:
        observation_dates = [datetime.datetime.strptime(
            el[0], '%Y%m%d').date() for el in data[0]]
    except ValueError as e:
        try:
            observation_dates = [datetime.datetime.strptime(
                el[0], '%Y%m').date() for el in data[0]]
        except ValueError as e:
            observation_dates = [datetime.datetime.strptime(
                el[0], '%Y').date() for el in data[0]]
    all_data_list.append(pd.Series(
        data=[el[1] for el in data[0]], index=pd.DatetimeIndex(observation_dates), name=name))
all_data_df = pd.concat(all_data_list, axis=1, join='outer')

In [ ]:
all_data_df = all_data_df.sort_index()
all_data_df = all_data_df.fillna(method='ffill')
all_data_df = all_data_df.loc[:, ~
                                     all_data_df.columns.duplicated()].copy()
all_data_df = all_data_df[~all_data_df[['Henry Hub Natural Gas Spot Price, Daily',
                                        'Natural Gas Futures Contract 1, Daily', 'Natural Gas Futures Contract 2, Daily']].isna().any(axis=1)]

In [ ]:
len(all_data_df.columns)

In [ ]:
correlations_df = all_data_df.corrwith(other=all_data_df['Natural Gas Futures Contract 1, Daily'])

In [ ]:
correlations_df.loc[(~correlations_df.index.str.contains('price')) & (~correlations_df.index.str.contains('Price')) & (~correlations_df.index.str.contains('Annual')) & ((correlations_df >= 0.5) | (correlations_df <= -0.5))]

In [ ]:
# Wyłapywać wyjątki i poszukać co tam jest źle
all_data_list = []
for name in df[df['series_id'].isin(ALL_IDS)]['name']:
    data = df[df['name'] == name]['data'].to_list()
    try:
        observation_dates = [datetime.datetime.strptime(
            el[0], '%Y%m%d').date() for el in data[0]]
    except ValueError as e:
        observation_dates = [datetime.datetime.strptime(
            el[0], '%Y%m').date() for el in data[0]]
    all_data_list.append(pd.Series(
        data=[el[1] for el in data[0]], index=pd.DatetimeIndex(observation_dates), name=name))
all_data_df = pd.concat(all_data_list, axis=1, join='outer')

In [ ]:
all_data_df = all_data_df.sort_index()
all_data_df = all_data_df.fillna(method='ffill')
all_data_df = all_data_df.loc[:, ~
                                     all_data_df.columns.duplicated()].copy()
all_data_df = all_data_df[~all_data_df[['Henry Hub Natural Gas Spot Price, Daily',
                                        'Natural Gas Futures Contract 1, Daily', 'Natural Gas Futures Contract 2, Daily']].isna().any(axis=1)]

In [ ]:
all_data_df = pd.concat([all_data_df, pd.DataFrame([
    [2.68, 2.68, 2.98, 3.43, 3.68],
    [2.71, 2.708, 2.98, 3.41, 3.66],
    [2.64, 2.64, 2.93, 3.34, 3.59]
    ], index=[datetime.datetime(2023, 9, 13), datetime.datetime(2023, 9, 14), datetime.datetime(2023, 9, 15)], columns=[
    'Henry Hub Natural Gas Spot Price, Daily',
    'Natural Gas Futures Contract 1, Daily',
    'Natural Gas Futures Contract 2, Daily',
    'Natural Gas Futures Contract 3, Daily',
    'Natural Gas Futures Contract 4, Daily'])], )

all_data_df[all_data_df.index >= datetime.datetime(2023, 9, 10)] = all_data_df[all_data_df.index >= datetime.datetime(2023, 9, 10)].fillna(method='ffill')

In [ ]:
all_data_df['Day of the week'] = all_data_df.index.dayofweek
all_data_df['Day'] = all_data_df.index.day
all_data_df['Month'] = all_data_df.index.month

In [ ]:
all_data_df = all_data_df[all_data_df.index >= datetime.datetime(2001, 1, 1)]

In [ ]:
pd.set_option('display.max_columns', None)
all_data_df.tail(10)

In [ ]:
all_data_df.corrwith(other=all_data_df['Natural Gas Futures Contract 1, Daily'])

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# all_data_df

In [ ]:
import seaborn as sns

for column in all_data_df.columns:
    sns.histplot(all_data_df[column])
    plt.show()

In [ ]:
from sklearn import preprocessing

columns_for_robust_scalling = ['Henry Hub Natural Gas Spot Price, Daily',
                               'Natural Gas Futures Contract 4, Daily',
                               'Natural Gas Futures Contract 3, Daily',
                               'Natural Gas Futures Contract 2, Daily',
                               'Natural Gas Futures Contract 1, Daily',
                               'U.S. Natural Gas Exports, Monthly']
other_columns = [
    column for column in all_data_df.columns if column not in columns_for_robust_scalling]
min_max_scaler = preprocessing.StandardScaler()
robust_scaler = preprocessing.RobustScaler(quantile_range=(0.0, 75.0))
robust_scaled_data = pd.DataFrame(robust_scaler.fit_transform(
    all_data_df[columns_for_robust_scalling]), columns=columns_for_robust_scalling, index=all_data_df.index)
min_max_scaled_data = pd.DataFrame(min_max_scaler.fit_transform(
    all_data_df[other_columns]), columns=other_columns, index=all_data_df.index)
all_data_df_scaled = pd.merge(robust_scaled_data, min_max_scaled_data, left_index=True, right_index=True)

In [ ]:
# all_data_df_scaled['Future 1 in one day'] = all_data_df['Natural Gas Futures Contract 1, Daily'].shift(
#     -1) - all_data_df['Natural Gas Futures Contract 1, Daily']
all_data_df_scaled['Future 1 in one day'] = all_data_df['Natural Gas Futures Contract 1, Daily'].shift(
    -1)

In [ ]:
all_data_df_scaled = all_data_df_scaled.fillna(0)

In [ ]:
import seaborn as sns

for column in all_data_df_scaled.columns:
    sns.histplot(all_data_df_scaled[column])
    plt.show()

In [ ]:
from IPython.display import Image

pp = sns.pairplot(data=all_data_df,
                  y_vars=['Natural Gas Futures Contract 1, Daily'],
                  x_vars=all_data_df.columns)
for ax in pp.axes.flat:
    ax.tick_params(labelrotation=45, labelsize=7)
    ax.set_xlabel(xlabel=ax.get_xlabel(), rotation=45, fontsize=8)
pp.savefig('Price correlations.png')

plt.clf()
Image(filename='Price correlations.png')

In [ ]:
# from IPython.display import Image
# import seaborn as sns
# import matplotlib.pyplot as plt 

# sns_plot = sns.pairplot(all_data_df_scaled, height=2.0)
# sns_plot.savefig("pairplot.png")

# plt.clf() # Clean parirplot figure from sns 
# Image(filename='pairplot.png') # Show pairplot as image

In [ ]:
from math import ceil, floor
import numpy as np
from sklearn.model_selection import train_test_split

# Podział na zbiór treningowy i walidacyjny

x_batches = np.array([all_data_df_scaled.iloc[i*50:].head(50).drop(columns=[
                     'Future 1 in one day']).values for i in range(floor(len(all_data_df_scaled)/50))])
y_batches = np.array([all_data_df_scaled['Future 1 in one day'].iloc[i*50:].head(
    50).values for i in range(floor(len(all_data_df_scaled)/50))])
y_batches = np.expand_dims(y_batches, -1)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_batches, y_batches, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5)

In [ ]:
import tensorflow as tf

def sign_metric(y_true, y_pred):
    return tf.reduce_mean(tf.cast((((y_true== y_pred) & (y_true==0)) | (y_true*y_pred>0)), tf.float32), axis=-1)

inputs = tf.keras.Input(shape=(x_train.shape[1], x_train.shape[2]))
x = tf.keras.layers.LSTM(256, return_sequences=True, stateful=False)(inputs)
x = tf.keras.layers.LSTM(64, return_sequences=True, stateful=False)(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=1e-4), loss="mae", metrics=[sign_metric])
history = model.fit(x_train, y_train, batch_size=8, epochs=1500, validation_data=(x_val, y_val), callbacks=[tf.keras.callbacks.EarlyStopping(patience=10), tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_sign_metric', save_best_only=True, save_weights_only=True, mode='max')])
model.load_weights('best_model.h5')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model.evaluate(x_test, y_test)
y_hat_test = model.predict(x_test)

In [ ]:
unrolled_y_test = np.array(y_test).ravel()
unrolled_y_hat_test = np.array(y_hat_test).ravel()
comparision_columns = list(zip(unrolled_y_test, unrolled_y_hat_test, np.roll(unrolled_y_test, -1), unrolled_y_test-unrolled_y_hat_test, (unrolled_y_test-unrolled_y_hat_test)/unrolled_y_test, unrolled_y_hat_test-np.roll(unrolled_y_test, -1), unrolled_y_test-np.roll(unrolled_y_test, -1)))
comparision_dataframe = pd.DataFrame(data=comparision_columns, columns=['true','prediction', 'previous price', 'difference','percent diff','predicted daily change','true daily change'])
display(comparision_dataframe.head(50))
# display(y_test[0,:,0])
# display(y_hat_test[0,:,0])

In [ ]:
comparision_dataframe.loc[(comparision_dataframe['percent diff'].abs() > 0.05) & (((comparision_dataframe['predicted daily change'] > 0) & (comparision_dataframe['true daily change'] < 0)) | ((comparision_dataframe['predicted daily change'] < 0) & (comparision_dataframe['true daily change'] > 0)))]

In [ ]:
plt.boxplot(comparision_dataframe['difference'], showfliers=False)
plt.show()

In [ ]:
# for column in all_data_df.columns:
#     plt.boxplot(all_data_df[column].dropna(), sym="*", labels=[column])
#     plt.show()